# 데이터 불러오기

In [73]:
import pandas as pd

In [74]:
master_df = pd.read_csv('./result/wholesale_2022-06-23_preprocessed.csv')

In [75]:
master_df

,category_code,p_code,p_group,kind_name_new,rank,unit,price,date
0,AW_A00115,AW_A00115,쌀,쌀(일반계)(20kg),상품,20kg,"44,500",2022-06-23
1,AW_A00116,AW_A00116,쌀,쌀(일반계)(20kg),중품,20kg,"41,000",2022-06-23
2,AW_A00153,AW_A00153,찹쌀,찹쌀(일반계)(40kg),중품,40kg,"97,800",2022-06-23
3,AW_A00198,AW_A00198,콩,흰 콩(국산)(40kg),상품,40kg,"240,000",2022-06-23
4,AW_A00200,AW_A00200,콩,흰 콩(수입)(35kg),중품,35kg,"122,000",2022-06-23
...,...,...,...,...,...,...,...,...
44564,SW_A00020,SW_A00020,건오징어,건오징어(20마리),중품,20마리,"77,000",2021-06-24
44565,SW_A00053,SW_A00053,김,마른김(1속),중품,1속,"6,000",2021-06-24
44566,SW_A00019,SW_A00019,건미역,건미역(1kg),상품,1kg,"10,000",2021-06-24
44567,SW_A00140,SW_A00140,전복,전복(1kg),중품,1kg,"33,000",2021-06-24


In [76]:
def remove_comma(price):
    """price 사이의 구두점 제거"""
    if type(price) == str:
        return "".join(price.split(","))
    else:
        return None

In [77]:
price_df = pd.DataFrame()
price_df['date'] = master_df['date']
price_df['price'] = master_df['price'].apply(remove_comma)
price_df['p_code'] = master_df['p_code']

In [78]:
price_df

,date,price,p_code
0,2022-06-23,44500,AW_A00115
1,2022-06-23,41000,AW_A00116
2,2022-06-23,97800,AW_A00153
3,2022-06-23,240000,AW_A00198
4,2022-06-23,122000,AW_A00200
...,...,...,...
44564,2021-06-24,77000,SW_A00020
44565,2021-06-24,6000,SW_A00053
44566,2021-06-24,10000,SW_A00019
44567,2021-06-24,33000,SW_A00140


# 일단 가격이 0인 데이터는 제거
    
    - 필요시 나중에 재검토 예정

In [79]:
wholesale_price_null_df = price_df[price_df['price'].isnull()]

In [80]:
wholesale_price_null_df

,date,price,p_code
56,2022-06-18,None,AW_A00115
57,2022-06-18,None,AW_A00116
58,2022-06-18,None,AW_A00153
59,2022-06-18,None,AW_A00198
60,2022-06-18,None,AW_A00200
...,...,...,...
44534,2021-06-26,None,SW_A00020
44535,2021-06-26,None,SW_A00053
44536,2021-06-26,None,SW_A00019
44537,2021-06-26,None,SW_A00140


In [81]:
wholesale_price_null_df.to_csv('./result/price_zero.csv', index=False)

In [82]:
# price_df 에서 price가 None인 index 제거
price_df.dropna(inplace=True)

In [83]:
price_df = price_df.reset_index(drop=True)

In [84]:
price_df

,date,price,p_code
0,2022-06-23,44500,AW_A00115
1,2022-06-23,41000,AW_A00116
2,2022-06-23,97800,AW_A00153
3,2022-06-23,240000,AW_A00198
4,2022-06-23,122000,AW_A00200
...,...,...,...
34483,2021-06-24,77000,SW_A00020
34484,2021-06-24,6000,SW_A00053
34485,2021-06-24,10000,SW_A00019
34486,2021-06-24,33000,SW_A00140


In [85]:
# 소매 물가 데이터 마지막 id = 146054 (2022-06-28 시점)
price_df['id'] = [ i + 146055 for i in range(len(price_df))]

In [86]:
price_df

,date,price,p_code,id
0,2022-06-23,44500,AW_A00115,146055
1,2022-06-23,41000,AW_A00116,146056
2,2022-06-23,97800,AW_A00153,146057
3,2022-06-23,240000,AW_A00198,146058
4,2022-06-23,122000,AW_A00200,146059
...,...,...,...,...
34483,2021-06-24,77000,SW_A00020,180538
34484,2021-06-24,6000,SW_A00053,180539
34485,2021-06-24,10000,SW_A00019,180540
34486,2021-06-24,33000,SW_A00140,180541


In [87]:
price_df.to_csv('./result/price_wholesale.csv', index=False)

# DB에 데이터 삽입하기

In [92]:
import pymysql

db = pymysql.connect(host="db-1.cnt4cbkbsgdt.us-east-1.rds.amazonaws.com",
                     port=3306,
                     user='admin',
                     passwd='12345678',
                     db='public_api',
                     charset='utf8')

cursor = db.cursor()

# 상품 데이터 삽입
for idx, row in price_df.iterrows():

    sql = f"INSERT INTO price VALUES({row['id']},'{row['date']}',{row['price']},'{row['p_code']}')"
    cursor.execute(sql)
    db.commit()

    if idx % 100 == 0:
        print(f"{idx} 번째 row insert 완료")

0 번째 row insert 완료
100 번째 row insert 완료
200 번째 row insert 완료
300 번째 row insert 완료
400 번째 row insert 완료
500 번째 row insert 완료
600 번째 row insert 완료
700 번째 row insert 완료
800 번째 row insert 완료
900 번째 row insert 완료
1000 번째 row insert 완료
1100 번째 row insert 완료
1200 번째 row insert 완료
1300 번째 row insert 완료
1400 번째 row insert 완료
1500 번째 row insert 완료
1600 번째 row insert 완료
1700 번째 row insert 완료
1800 번째 row insert 완료
1900 번째 row insert 완료
2000 번째 row insert 완료
2100 번째 row insert 완료
2200 번째 row insert 완료
2300 번째 row insert 완료
2400 번째 row insert 완료
2500 번째 row insert 완료
2600 번째 row insert 완료
2700 번째 row insert 완료
2800 번째 row insert 완료
2900 번째 row insert 완료
3000 번째 row insert 완료
3100 번째 row insert 완료
3200 번째 row insert 완료
3300 번째 row insert 완료
3400 번째 row insert 완료
3500 번째 row insert 완료
3600 번째 row insert 완료
3700 번째 row insert 완료
3800 번째 row insert 완료
3900 번째 row insert 완료
4000 번째 row insert 완료
4100 번째 row insert 완료
4200 번째 row insert 완료
4300 번째 row insert 완료
4400 번째 row insert 완료
4500 번째 row insert 완료
